In [1]:
from sklearn import cross_validation
from sklearn.datasets import load_boston

import pandas as pd
import numpy as np

from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import mean_squared_error


boston = load_boston()
X=pd.DataFrame(boston.data[:100,:], columns=boston.feature_names)
y=boston.target[:100]
x=X['LSTAT'].values

np.random.seed(0) # 乱数のシード設定
indices = np.random.permutation(len(x))
x = x[indices] # データの順序をランダムに並び替え
y = y[indices]
n_fold = 10 # 交差検定の回数
k_fold = cross_validation.KFold(n=len(x),n_folds = n_fold)

dim=np.arange(1,11)# グリッドサーチの候補となるハイパーパラメータを考えます。次元は１～１1までを調べます。
lambda_=np.arange(1,30,2)/1e5#ハイパーパラメータを格納しています。

i = 0
L=len(dim)*len(lambda_)
score = np.zeros((L,3))
tmp=[]
for d in dim:
    degree_=PolynomialFeatures(degree=d)
    for l in lambda_:
        lin_ = Ridge(normalize=True,alpha=l)
        for train, test in k_fold:
            x_=degree_.fit_transform(x[train,None])
            x_test=degree_.fit_transform(x[test,None])
            #normalize=Trueは入力データを正規化してトレーニングすることを意味します。
            lin_.fit(x_,y[train])

            tmp.append(mean_squared_error(y[test],lin_.predict(x_test)))

        score[i,0] = d
        score[i,1] = l
        score[i,2] = sum(tmp) / len(tmp)
        i+=1
        tmp=[]

print(score[np.argmin(score[:,-1])])

C:\Users\suzukiry\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[  5.00000000e+00   9.00000000e-05   1.07818939e+01]
